In [9]:
import urllib.request
from collections import defaultdict, namedtuple
from itertools import combinations, chain
import re
import numpy as np
from heapq import heappop, heappush

# It's day 11
day = 11

# No need to take from web today, just write the input hard
# It's my very first time resolving this kind of problems with code, so i'm
# going to refer totally to @norvig to learn something new!

In [37]:
def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def step(prev, state):
  return ([] if state is None else step(prev, prev[state]) + [state])

def heuristic(state):
  # Calculating how many more moves we need to finish (heuristic obviously)
  # The idea is that we can move 2 items at the same time and we count how many
  # floors needs every item to reach top floor
  remaining = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
  return np.ceil(remaining / 2)

def moves(state):
  # Possible moves from actual state
  lvl, floors = state

  # Elevator can move 1 level at a time
  for new_lvl in {min(lvl + 1, 3), max(lvl - 1, 0)}:
    # We can bring two things at most
    for stuff in obj_to_move(floors[lvl]):
      new_floors = tuple(
          (s | stuff if i == new_lvl else # Adding new stuff to the level we are heading 
           s - stuff if i == state.elevator else # Removing stuff taken from this level
           s) # Unchanged level cause we are moving in other floors
           for (i, s) in enumerate(state.floors)
           )
      # Check if the stuff we are moving isn't destroying the building
      if legal_floor(new_floors[lvl]) and legal_floor(new_floors[new_lvl]):
        yield State(new_lvl, new_floors)

def legal_floor(floor):
  # Checking if the floor isn't going to implode
  # Generators in this floor
  gens = any(g.endswith('G') for g in floor)
  # Chips in this floor
  chips = [c for c in floor if c.endswith('M')]
  # A floor is legal if there's no generator on it
  # or if every chip is connected to its generator
  return not gens or all(c[0] + 'G' in floor for c in chips)


def obj_to_move(objs):
  for x in chain(combinations(objs, 1), combinations(objs, 2)):
    yield frozenset(x)

# SETUP
# Frozensets are still dark-code for me
def fs(*items): return frozenset(items)

State = namedtuple('State', 'elevator, floors')

floors = {0, 1, 2, 3}

In [40]:
# PART ONE
start = State(0, (fs('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM'),
                  fs('PM', 'pM'),
                  fs(),
                  fs()))

path = problem_solver(start, heuristic, moves)
print(f'Response is the length of the path to the top: {len(path) -1}')

Response is the length of the path to the top: 47


In [41]:
# PART TWO
start  = State(0, (fs('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM', 'EG', 'EM', 'DG', 'DM'),
                  fs('PM', 'pM'),
                  fs(),
                  fs()))

path = problem_solver(start, heuristic, moves)
print(f'Response is the length of the path to the top: {len(path) -1}')

Response is the length of the path to the top: 71
